In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re
import sys
import os
sys.path.append(os.path.abspath(".."))
from utils.utils import *

In [8]:
year = 2021
reporting_data_path = '/share/garg/311_data/sb2377/clean_codebase/data_{}.csv'.format(year)
save_path = '/share/garg/311_data/sb2377/clean_codebase/{}_weekly_report_data_tracts.csv'.format(year)

In [9]:
df = pd.read_csv(reporting_data_path)

/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_940503/3063820325.py:1: DtypeWarning: Columns (9,21,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(reporting_data_path)


In [4]:
len(df)

3225185

In [7]:
len(df)

3169777

In [10]:
len(df)

3220901

In [11]:
3225185 + 3169777 + 3220901

9615863

In [5]:
# get type information
def concatenate_and_clean(t):
    concatenated = ''.join(t)
    # Remove all characters that are not letters
    cleaned = re.sub(r'[^A-Za-z]', '', concatenated)
    return cleaned

df['typeagency'] = list(zip(df['Complaint Type'], df['Agency']))
df['typeagency'] = df['typeagency'].apply(concatenate_and_clean)

In [4]:
census_gdf, final_graph, census_gdf_raw = generate_graph_census()

Using the default year of 2021
Using FIPS code '36' for input 'NY'


HTTPError: HTTP Error 403: Forbidden

In [7]:
# only get rows with valid location information
complaints_nonan_df = df[~(df.Latitude.isna()|df.Longitude.isna())]
complaints_gdf = gpd.GeoDataFrame(complaints_nonan_df,
                                  geometry=gpd.points_from_xy(complaints_nonan_df.Longitude, complaints_nonan_df.Latitude),
                                  crs='EPSG:4326')

# get census block information
reports_df = gpd.sjoin(complaints_gdf.to_crs('EPSG:2263').reset_index(drop=True),census_gdf.to_crs('EPSG:2263'), how='inner', op='intersects')

# get date information
reports_df['Created Date'] = pd.to_datetime(reports_df['Created Date'])
reports_df['report_week'] = reports_df['Created Date'].dt.isocalendar().week
reports_df['month'] = reports_df['Created Date'].dt.month
reports_df.loc[(reports_df['month'] == 1) & (reports_df['report_week'] > 50), 'report_week'] = 0

# get type information
def concatenate_and_clean(t):
    concatenated = ''.join(t)
    # Remove all characters that are not letters
    cleaned = re.sub(r'[^A-Za-z]', '', concatenated)
    return cleaned

reports_df['typeagency'] = list(zip(reports_df['Complaint Type'], reports_df['Agency']))
reports_df['typeagency'] = reports_df['typeagency'].apply(concatenate_and_clean)

NameError: name 'census_gdf' is not defined

In [6]:
# create num census block x num type x num weeks full T dataset
census_blocks = np.array(census_gdf['GEOID'].unique())
types = np.array(reports_df['typeagency'].unique())
weeks = reports_df['report_week'].unique()

# get all pairs of node x type
pairs = np.array(np.meshgrid(census_blocks,types)).T.reshape(-1,2)
pairs_df = pd.DataFrame()
pairs_df['GEOID'] = pairs[:,0]
pairs_df['typeagency'] = pairs[:,1]

# add in type and agency information
type_df = reports_df[['typeagency', 'Complaint Type', 'Agency']].drop_duplicates()
type_df = type_df[type_df['Complaint Type'] != 'Litter Basket / Request']
pairs_df = pd.merge(pairs_df, type_df, on='typeagency')

dfs = []

for i in weeks:
    data = pairs_df.copy()
    data['report_week'] = i
    dfs.append(data)
    
all_data = pd.concat(dfs)

# add in T labels
# set T = 1 for instances where there is a complaint
reported_data = reports_df[['GEOID', 'typeagency', 'report_week']].drop_duplicates()
reported_data['reported'] = 1
binary_reports_df = pd.merge(all_data, reported_data, on=['GEOID', 'typeagency', 'report_week'], how='left')
binary_reports_df['report_week_of_year'] = binary_reports_df['report_week']
binary_reports_df['reported'] = binary_reports_df['reported'].fillna(0)

In [7]:
assert(len(census_blocks) * len(types) * len(weeks) == len(binary_reports_df))

In [8]:
len(binary_reports_df)

27367716

In [9]:
assert(len(binary_reports_df[binary_reports_df.isna().any(axis=1)]) == 0)

In [10]:
binary_reports_df.to_csv(save_path)